<a href="https://colab.research.google.com/github/m10923039/-/blob/main/CNN_talos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install talos

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 9.5MB 7.3MB/s 
     |████████████████████████████████| 5.4MB 27.3MB/s 
     |████████████████████████████████| 2.6MB 40.3MB/s 
  Created wheel for chances: filename=chances-0.1.9-cp37-none-any.whl size=41610 sha256=e294a2d2b2d347daf556170d38a6de5776185a8a38228402d24dd25ebd134e13
  Stored in directory: /root/.cache/pip/wheels/75/33/46/c871b94249bd57d17797d049b3dff8e3a09c315afb67eb14c6
  Created wheel for wrangle: filename=wrangle-0.6.7-cp37-none-any.whl size=49894 sha256=215d87f5fc5bdcdeec9a3bd664d2d0e92f4c2d642642e13e7efde7212955b938
  Stored in directory: /root/.cache/pip/wheels/bf/1b/50/d0403ce6ef269e364894da7b50db68db14c4ac62c577561e2d
  Created wheel for kerasplotlib: filename=kerasplotlib-0.1.6-cp37-none-any.whl size=3603 sha256=4f7478ef588db0ffa00b96797b7ae864f987891e57b854f33b3249848d155f77
  Stored in directory: /root/.cache/pip/wheels/9d/d3/8c/9503a22b0a38e8b21c70ad834e4606d209193443e

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,Conv2D,Activation,BatchNormalization,MaxPooling2D
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split 

# import talos
# from talos.utils.gpu_utils import multi_gpu

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 讀取 cifar10 檔案
((train_feature, train_label), (test_feature, test_label)) = cifar10.load_data()

# MinMax 將 像素縮放到0~1
train_feature_vector = train_feature / 255

# onehot 編碼
train_label_onehot = np_utils.to_categorical(train_label)

# 切分 validation_data ，可以不做
(x_train,x_val,y_train,y_val) =train_test_split(train_feature_vector, train_label_onehot, test_size = 0.2, random_state = 0)

# 測試 資料的預處理
test_feature_vector = test_feature / 255
test_label_onehot = np_utils.to_categorical(test_label)

# 各個變數的 shape
print( 'train feature datas =', train_feature.shape )
print( 'test feature datas =', test_feature.shape )
print( 'x_train =', x_train.shape )
print( 'y_train =', y_train.shape )
print( 'x_val =', x_val.shape )
print( 'y_val =', y_val.shape )


170500096/170498071 [==============================] - 2s 0us/step
train feature datas = (50000, 32, 32, 3)
test feature datas = (10000, 32, 32, 3)
x_train = (40000, 32, 32, 3)
y_train = (40000, 10)
x_val = (10000, 32, 32, 3)
y_val = (10000, 10)


# 測試pre_model

In [ ]:
# 選取 pre_model 的層數
test_model = VGG16(include_top=False, weights='imagenet', input_tensor=None,
      input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3])
    )

In [ ]:
test_model.save('VGG16.h5')

In [ ]:
test_model.summary()

In [ ]:
test_model = Model(inputs=test_model.input, outputs=test_model.get_layer('block3_conv2').output)
#  Model(inputs=net.input, outputs=output_layer)
len(test_model.layers)

9

# **TPU準備**
載入 TPU 程式

In [ ]:
import os
tf.keras.backend.clear_session()
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.experimental.TPUStrategy(resolver)

# 模型 model

In [ ]:
def minimal(x1,y1,x2,y2):
                              # l1 = tf.keras.regularizers.l1(0.01)
                              # p = {'activation':['relu', 'elu'],
                              #       'optimizer': ['Nadam', 'Adam'],
                              #       'losses': ['categorical_crossentropy'],
                              #       'batch_size': [64,128,256],
                              #       'epochs': [60],
                              #       'node_1' : [64,256,1024],
                              #       'node_2' : [64,128,256],
                              #       'regularizers' :[l2,l1]
                              #     }
# p 設定要搜索的參數 字典
# 
# 
  l2 = tf.keras.regularizers.l2(0.01)
  p = {'activation':['relu'],
      'optimizer': ['Adam'],
      'losses': ['categorical_crossentropy'],
      'batch_size': [1500,3000],
      'epochs': [60],
      'node_1' : [256,384],
      'node_2' : [256,384],
      'regularizers' :[l2]
    }
      
  def create_model(params):
      net = VGG16(include_top=False, weights='imagenet', input_tensor=None,
              input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3])
              )
      
      # 透過layer_name 節取 model的某一層
      layer_name='block3_conv3'

      # 設定取到該層的都使用預設參數，不訓練參數
      layer_index=net.layers.index(net.get_layer(layer_name))
      
      # 建立模型
      base_model = net.get_layer(layer_name).output
      base_model = tf.keras.layers.Conv2D(128, (1, 1), padding='valid')(base_model)
      base_model = tf.keras.layers.MaxPooling2D( pool_size = (3, 3) )(base_model)
      
      # 攤平 feature map 
      base_model = Flatten()(base_model)
      

      base_model = Dropout(0.3)(base_model)
      base_model = tf.keras.layers.BatchNormalization()(base_model)
      base_model = Dense(units = params['node_1'], kernel_initializer = tf.keras.initializers.GlorotNormal()
                 ,kernel_regularizer= params['regularizers'] , activation= params['activation'])(base_model)


      base_model = Dense(units = params['node_2'], kernel_initializer = tf.keras.initializers.GlorotNormal()
                 ,kernel_regularizer = params['regularizers'] , activation= params['activation'])(base_model)

      base_model = Dropout(0.3)(base_model)
      output_layer = Dense(y_train.shape[1], activation='softmax', name='softmax')(base_model)

      # 設定凍結與要進行訓練的網路層
      net_final = Model(inputs=net.input, outputs=output_layer)

      for layer in net_final.layers[:layer_index]:
          layer.trainable = False

      return net_final
  ###########################################################################################     
  def iris_model(x_train, y_train, x_val, y_val, params):

    with strategy.scope():
      model = create_model(params)
      precision=tf.keras.metrics.Precision(name='precision')
      recall=tf.keras.metrics.Recall(name='recall')
      accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')
      model.compile(optimizer=params['optimizer'],
              loss=params['losses'],
              metrics = [recall,precision,accuracy],
             
              )
      print(model.summary())
    callbacks_list=[
      tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',mode='min', patience=10 ,restore_best_weights=True
      )]
      
    out = model.fit(x1, y1,
          batch_size=params['batch_size'],
          epochs=params['epochs'],
          validation_data=(x2,y2),
          verbose=2,
          callbacks=callbacks_list                
            )
    return out, model
  ###########################################################################################
  # with tf.keras.utils.CustomObjectScope():
  scan_object = talos.Scan(x=x1,
                y=y1,
                x_val=x2,
                y_val=y2,
                model=iris_model,
                params=p,
                experiment_name='VGG16')
    
  return scan_object

# 訓練

In [ ]:
from talos import Deploy
import datetime

# 配合儲存模型，擷取時間
def getTime():
  dt0 = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc) # 展現獲取0時區時間
  dt8 = dt0.astimezone(datetime.timezone(datetime.timedelta(hours=8)))  # 轉換時區到東八區
  dt0 = datetime.datetime.strftime(dt0,'%m%d%H%M')
  dt8 = datetime.datetime.strftime(dt8,'%m%d%H%M')
  return dt8

#  開始訓練
scan_object=minimal(x_train,y_train,x_val,y_val)

#  儲存模型
file_name ="VGG16_{}".format(getTime())
Deploy(scan_object,file_name ,metric='val_recall') 

In [ ]:
# /content/VGG16_04212053.zip
Deploy(scan_object,'VGG16_04212053_1' ,metric='val_accuracy') 

Deploy package VGG16_04212053_1 have been saved.
data is not 2d, dummy data written instead.


In [ ]:
# access the summary details
print(scan_object.details)
print(scan_object.data)

In [ ]:
scan_object

In [ ]:
best_model = scan_object.data[scan_object.data['val_recall']==scan_object.data['val_recall'].max()]
best_model

,start,end,duration,round_epochs,loss,recall,precision,accuracy,val_loss,val_recall,val_precision,val_accuracy,activation,batch_size,epochs,losses,node_1,node_2,optimizer,regularizers
6,04/21/21-120124,04/21/21-120228,63.788911,34,0.23628,0.9547,0.974035,0.9653,0.675733,0.8052,0.851703,0.8241,relu,3000,60,categorical_crossentropy,384,256,Adam,<tensorflow.python.keras.regularizers.L2 objec...


In [ ]:
precision=tf.keras.metrics.Precision(name='precision')
recall=tf.keras.metrics.Recall(name='recall')
accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')
best_model.compile(optimizer='Adam',
        loss='categorical_crossentropy',
        metrics = [recall,precision,accuracy],
        )
eval = best_model.evaluate( test_feature_vector, test_label_onehot )

## 回復模型預測

In [ ]:
"""
此方法 CNN　會有問題

ERROE
Deploy package VGG16_04212053_1 have been saved.
data is not 2d, dummy data written instead.
<talos.commands.deploy.Deploy at 0x7f6a391c3710>

# restore 回復模型預測
# from talos import Restore
restore = Restore('/content/drive/MyDrive/Colab Notebooks/VGG16_04212003.zip')
"""


In [ ]:
"""
CNN 適用此方法 恢復模型
1. 找到被儲存的模型，Deploy的模型
2. 使用以下方是讀取模型
3. 路徑名稱到 _model 前
"""
import talos
from talos.utils.load_model import load_model
# /content/VGG16_04212003/VGG16_04212003_model.h5
pre_model = load_model('/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/VGG16_04212003/VGG16_04212003' + '_model')

In [ ]:
pre_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [ ]:
# restore 回復模型預測
# restore.results
precision=tf.keras.metrics.Precision(name='precision')
recall=tf.keras.metrics.Recall(name='recall')
accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')

# results_best_model = restore.results[restore.results['f1score']==restore.results['f1score'].max()]
results_best_model = pre_model
results_best_model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics = [precision,recall,talos.utils.metrics.f1score,accuracy]
                    )


preds=results_best_model.evaluate(test_feature_vector,test_label_onehot)
print('loss ： %.5f' % preds[0])
print('Precision：%.5f' % preds[1])
print('Recall：%.5f' % preds[2])
print('f1score：%.5f' % preds[3])
print('accuracy： %.5f' %preds[4])

313/313 [==============================] - 69s 216ms/step - loss: 0.6714 - precision: 0.8465 - recall: 0.7965 - f1score: 0.8201 - accuracy: 0.8164
loss ： 0.6921972632408142
Precision： 0.8410032987594604
Recall： 0.7912999987602234
f1score： 0.8149928450584412
accuracy 0.8119999766349792


In [ ]:
results_best_model.history.history

{}

In [ ]:
print('loss ： %.6f' % preds[0])
print('Precision：%.6f' % preds[1])
print('Recall：%.6f' % preds[2])
print('f1score：%.6f' % preds[3])
print('accuracy： %.6f' %preds[4])

loss ： 0.692197
Precision：0.841003
Recall：0.791300
f1score：0.814993
accuracy： 0.812000



313/313 [==============================] - 11s 32ms/step 
- loss: 0.6714 - precision: 0.8465 - recall: 0.7965 - f1score: 0.8201  accuracy: 0.8164

---
- loss ： 0.6921972036361694
- Precision： 0.8410032987594604
- Recall： 0.7912999987602234
- f1score： 0.8149928450584412
- accuracy 0.8119999766349792

---








In [ ]:
best_model1 = scan_object.data[scan_object.data['val_recall']==scan_object.data['val_recall'].max()]
best_model1
